<a href="https://colab.research.google.com/github/chueneelvin/Databricks/blob/main/PDF_QnA_with_Langchain_and_OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install required packages

In [ ]:
!pip -q install langchain pypdf langchain-community langchain-text-splitters langchain_experimental langchain_openai langchain-chroma langchain-pinecone python-dotenv chromadb faiss-cpu unstructured[pdf] poppler-utils langsmith tesseract

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 MB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 46.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5

# Importing the dependecies

In [ ]:
from langchain_community.document_loaders import PyPDFLoader        # Loading the docuements
from langchain_community.document_loaders import DirectoryLoader   # Loading the documents from a directory
from langchain_text_splitters import RecursiveCharacterTextSplitter # Text chunks using recursive spliter
from langchain_experimental.text_splitter import SemanticChunker    # Semantic text chuncking
from langchain_openai import OpenAIEmbeddings                       # openai embedding models
from langchain_chroma import Chroma                                 # vector database Chromadb
from langchain.vectorstores import Pinecone                  # vector database Pinecone
from langchain_community.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain_openai import OpenAI
import pinecone
import os
from langchain.chains import StuffDocumentsChain


# Loading the data

## loading single files (pdf)

In [ ]:
loader = PyPDFLoader("/content/Farming Potatoes in South Africa_ What You Need to Know.pdf")
docs = loader.load()

In [ ]:
docs[0]

Document(metadata={'source': '/content/Farming Potatoes in South Africa_ What You Need to Know.pdf', 'page': 0}, page_content="\uf1bf\nFarming P otatoes In South Africa: What\nYou Need To Know\n“My idea of hea ven is a gr eat big bak ed potat o and\nsomeone t o shar e it with. ”\n- Opr ah Winfr ey\nWhat's in this guide?\n1. Introduction: Farming potatoes in South Africa\n2. All about seed potatoes\n3. Growing potatoes in South Africa\n4. Challenges of potato farming\n5. Sustainable potato farming\nChapter 1\nIntroduction: F arming P otatoes in South Africa\nIn South Africa, maiz e meal and br ead ar e the most commonly consumed sour ces of\ncarbohy drates. Howe ver, South Africans ha ve eaten twice as many potat oes o ver the past\ndecade compar ed to the decade befor e it, buo yed b y a gr owing middle class.")

# Text Chunking

## Recursive chuncking

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=15
)

chunked_docs = text_splitter.split_documents(docs)
len(chunked_docs)

54

In [ ]:
chunked_docs[:3]

[Document(metadata={'source': '/content/Farming Potatoes in South Africa_ What You Need to Know.pdf', 'page': 0}, page_content="\uf1bf\nFarming P otatoes In South Africa: What\nYou Need To Know\n“My idea of hea ven is a gr eat big bak ed potat o and\nsomeone t o shar e it with. ”\n- Opr ah Winfr ey\nWhat's in this guide?\n1. Introduction: Farming potatoes in South Africa\n2. All about seed potatoes\n3. Growing potatoes in South Africa\n4. Challenges of potato farming\n5. Sustainable potato farming\nChapter 1\nIntroduction: F arming P otatoes in South Africa"),
 Document(metadata={'source': '/content/Farming Potatoes in South Africa_ What You Need to Know.pdf', 'page': 0}, page_content='In South Africa, maiz e meal and br ead ar e the most commonly consumed sour ces of\ncarbohy drates. Howe ver, South Africans ha ve eaten twice as many potat oes o ver the past\ndecade compar ed to the decade befor e it, buo yed b y a gr owing middle class.'),
 Document(metadata={'source': '/content/Farm

## Semantic chunking

In [ ]:
from langchain_experimental.text_splitter import SemanticChunker

text_splitter1 = SemanticChunker(OpenAIEmbeddings())
text_splitter1

In [ ]:
semantic_chunk_docs = text_splitter1.split_documents(docs)
semantic_chunk_docs[:3]

In [ ]:
len(semantic_chunk_docs)

# Convert chunks into embeddings and store into FAISS vector store

In [ ]:
# Get the API key from user data
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')

In [ ]:
LANGCHAIN_TRACING_V2=True
LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
LANGCHAIN_API_KEY="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"
LANGCHAIN_PROJECT="default"

In [ ]:
# Initialising the embeddings model

embeddings_model = OpenAIEmbeddings(model = "text-embedding-3-small")
embeddings_model

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x7cfd327676d0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7cfd327163e0>, model='text-embedding-3-small', dimensions=None, deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

## Recursive Character Splitting for Chunks

In [ ]:
faiss_db = FAISS.from_documents(chunked_docs, embeddings_model)
faiss_db

In [ ]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")
#from langchain.prompts import PromptTemplate
#from langchain.chains import LLMChain
#llm = OpenAI()
#chain = StuffDocumentsChain(llm_chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template("Summarize this content: {context}")), verbose=True)
#chain

<ipython-input-15-2c8a90fa5f3d>:1: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/v0.2/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/v0.2/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/v0.2/docs/how_to/#qa-with-rag
  chain = load_qa_chain(OpenAI(), chain_type="stuff")


In [ ]:
query= "when to plant potatoes for max yield?"
results = faiss_db.similarity_search(query)
chain.invoke({'input_documents': results, 'question': query})['output_text']

' The best time to plant potatoes for maximum yield is typically in the spring, from August to early June in frost-free areas or from August to December into early January in areas with frost. This ensures that the potatoes have enough time to develop and grow before harvest. However, it is also dependent on the specific region and its climate, as well as the specific requirements of the potato variety being planted.'

## Semantic Character splitting for document chunking

# Persist the vector embeddings into a  vector database

## ChromaDB

(install via pip : pip install langchain-chroma)

## Pinecone

### Initialize the pinecone vector db

In [ ]:
from langchain_pinecone import PineconeVectorStore
index_name = "chain"
PineconeVectorStore.from_documents(chunked_docs,embeddings_model,index_name=index_name)


In [ ]:
pinecone_vectorstore = PineconeVectorStore(embedding=embeddings_model,index_name=index_name)

In [ ]:
pinecone_results = pinecone_vectorstore.similarity_search(query)
chain.invoke({'input_documents': pinecone_results, 'question': query})['output_text']

' The best time to plant potatoes for maximum yield would be in the spring, from August to early June the following year in frost-free areas or from August to December/January in areas with frost. This allows for slow development of tubers and cooler growing conditions, which are ideal for potato production.'

In [ ]:
# Deleting the pinecone index
from pinecone import Pinecone

pc = Pinecone(api_key='8c3ac13f-780e-4b95-984e-90be091e60e7')
pc.delete_index('chain')

In [ ]:
pc

## Single store